# Initial file upload: TMDB_tv_dataset_v3.csv

In [ ]:
import pandas as pd

df = pd.read_csv(r"TMDB_tv_dataset_v3.csv")

In [ ]:
import matplotlib as plt
%matplotlib inline

import numpy as np
import seaborn as sb
import warnings

# Step 1: Data Preparation



In [ ]:
# To have an overview of the dataset
df.describe()

In [ ]:
# To have an overview of missing data and types of data

# ניצור טבלת סיכום מקיפה לכל עמודה
summary = pd.DataFrame({
    'non_null_count': df.notnull().sum(),
    'null_percent': df.isnull().mean() * 100,
    'num_unique': df.nunique(),
    'data_type': df.dtypes,
}).sort_values(by='null_percent', ascending=False)

print(summary)

In [ ]:
df.head()

In [ ]:
# Show all columns
pd.set_option('display.max_columns', None)
print(df.head(10))

## Look at rows and columns - Rows and columns duplication, check nulls and unknown - to

---

remove them

In [ ]:
# חיפוש עמודות זהות - לא נמצאו
# To see columns with duplicate content - Didn't find any!
# יצירת רשימה לאחסון זוגות עמודות עם תוכן כפול
duplicate_content = []

# לולאה על כל זוגות עמודות
for i in range(len(df.columns)):
    for j in range(i+1, len(df.columns)):
        col1 = df.columns[i]
        col2 = df.columns[j]
        if df[col1].equals(df[col2]):
            duplicate_content.append((col1, col2))

print("Columns with duplicate content:", duplicate_content)

In [ ]:
# Screen for duplication whole row - by three parameters: id, name, original_name
# חיפוש שורות עם אותו id, name, original_name - נמצאו ובהמשך נמחק אחד מהם
# לפי שלושת הפרמטרים האלה כי זה חוסך זמן הרצה
# ----------------------------
# חלק 1 – בדיקה והצגת כפילויות
# ----------------------------

# 1️⃣ מציאת כל השורות הכפולות לפי עמודות ספציפיות
duplicate_rows = df[df.duplicated(subset=['id', 'name', 'original_name'], keep=False)]

# 2️⃣ מיון התוצאות כדי שיהיה קל לראות את הכפילויות
duplicate_rows = duplicate_rows.sort_values(by=['id', 'name', 'original_name'])

# 3️⃣ הצגת 10 דוגמאות ראשונות של כפילויות
print("דוגמאות של כפילויות:")
print(duplicate_rows.head(10))

# 4️⃣ הדפסת מספר השורות הכולל שמזוהות ככפולות
num_duplicates = len(duplicate_rows)
print(f"\nנמצאו {num_duplicates} שורות כפולות (כולל כל העותקים).")

In [ ]:
# חלק 2 – הסרת כל הכפילויות

# Combine repeated rows into one - New df copy for this process: df_unique

# מספר השורות המקורי
original_rows = len(df)

# ----------------------------
# הסרת כל הכפילויות, שמירה על מופע אחד לכל שילוב של ['id', 'name', 'original_name']
# ----------------------------
df_unique = df.drop_duplicates(subset=['id', 'name', 'original_name']).reset_index(drop=True)

# חישוב מספר השורות הלא ייחודיות שהוסרו
removed_rows = original_rows - len(df_unique)

# הצגה מהירה של התוצאה
print("\nתוצאות לאחר הסרת כפילויות:")
print(df_unique.head())
print(f"סה\"כ שורות ייחודיות: {len(df_unique)}")
print(f"סה\"כ שורות לא ייחודיות שנמחקו: {removed_rows}")

# New data file name: df_unique

In [ ]:
print(f"מספר העמודות: {len(df_unique.columns)}")
print("שמות העמודות:")
print(df_unique.columns)

# Explore target - popularity

In [ ]:
df_unique['popularity'].describe()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

plt.figure(figsize=(10, 6))
sns.boxplot(x=np.log1p(df['popularity']))  # נתונים מופיעים על ציר X
plt.title('Boxplot of log(Popularity) - Horizontal', fontsize=14, weight='bold')
plt.xlabel('log(Popularity + 1)')
plt.show()


In [ ]:
"""
# לא שייך לכאן!!!!!!!!!!!!!!!1

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# אם אין עדיין עמודת קטגוריה, אפשר ליצור:
labels = ['Low', 'Mid', 'Hit']
df['popularity_3cat'] = pd.qcut(df['popularity'], q=3, labels=labels)

# יוצרים עמודת לוגריתם זמנית
df['log_popularity'] = np.log1p(df['popularity'])

plt.figure(figsize=(12, 6))
sns.boxplot(x='log_popularity', y='popularity_3cat', data=df, palette='Set2')
plt.title('Boxplot of log(Popularity) by Category', fontsize=14, weight='bold')
plt.xlabel('log(Popularity + 1)')
plt.ylabel('Popularity Category')
plt.show()
"""

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(df['popularity'], bins=30, kde=True, color='skyblue')
plt.title('Histogram of Popularity', fontsize=14, weight='bold')
plt.xlabel('Popularity')
plt.ylabel('Count')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

plt.figure(figsize=(10, 6))
sns.histplot(np.log1p(df['popularity']), bins=30, kde=True, color='skyblue')
plt.title('Histogram of log(Popularity + 1)', fontsize=14, weight='bold')
plt.xlabel('log(Popularity + 1)')
plt.ylabel('Count')
plt.show()


In [ ]:
pop_max = df['popularity'].quantile(0.99)
plt.figure(figsize=(10, 6))
sns.histplot(df[df['popularity'] <= pop_max]['popularity'], bins=30, kde=True, color='skyblue')
plt.title('Histogram of Popularity (99th percentile)', fontsize=14, weight='bold')
plt.xlabel('Popularity')
plt.ylabel('Count')
plt.show()


In [ ]:
# Check overall nulls
import missingno as msno
msno.matrix(df_unique)

In [ ]:
# Checking nulls by percentage
missing_percent = df_unique.isnull().mean() * 100
print(missing_percent.sort_values(ascending=False))

# Repeated columns with the same information
## Columns handeling - unit or remove duplicate

In [ ]:
# Repeated columns with the same information
# Second - TV show name and original name

print(df_unique[['name', 'original_name']].head(20))

different_rows = df_unique[df_unique['name'] != df_unique['original_name']]
print(different_rows[['name', 'original_name']].head(20))

In [ ]:

# Coulmns 'name' and 'original_name' - are mostly the same.
# Check for missing values in 'name' column - without the name the row is useless
# ספירת שורות שבהן 'name' חסר

# מציאת השורות שבהן 'name' חסר
missing_name_rows = df_unique[df_unique['name'].isnull()]
missing_name_count = len(missing_name_rows)

print(f"מספר השורות שבהן 'name' חסר: {missing_name_count}")
missing_name_rows[['id', 'name', 'original_name']].head(20)  # תצוגה לדוגמה


In [ ]:
import numpy as np

# הגדרת תנאי לחסר אמיתי בשם:
# 1. name הוא NaN
# 2. name הוא ריק ("")
# 3. name מכיל רק נקודות, כמו "..." או "....."
missing_name_condition = (
    df_unique['name'].isna() |
    (df_unique['name'].astype(str).str.strip() == "") |
    (df_unique['name'].astype(str).str.match(r'^\.*$'))
)

# שליפת השורות החסרות לפי התנאי
missing_name_rows = df_unique[missing_name_condition]

print(f"מספר השורות שבהן name חסר או בעייתי: {len(missing_name_rows)}")
missing_name_rows[['id', 'name', 'original_name']].head(20)  # תצוגה לדוגמה


In [ ]:
# מספר שורות לפני המחיקה
rows_before = len(df_unique)
print(f"סה\"כ שורות לפני המחיקה: {rows_before}")

# תנאי לזיהוי name לא תקין: NaN, ריק, או רק נקודות
missing_name_condition = (
    df_unique['name'].isna() |
    (df_unique['name'].astype(str).str.strip() == "") |
    (df_unique['name'].astype(str).str.match(r'^\.*$'))
)

# שמירת השורות למחיקה (לצפייה או שמירה)
missing_name_rows = df_unique[missing_name_condition]
deleted_ids = missing_name_rows['id'].tolist()

# מחיקת השורות הלא תקינות
df_unique = df_unique[~missing_name_condition].reset_index(drop=True)

# מספר שורות אחרי המחיקה
rows_after = len(df_unique)
print(f"סה\"כ שורות אחרי המחיקה: {rows_after}")

# כמה שורות נמחקו
removed_rows = rows_before - rows_after
print(f"סה\"כ שורות שנמחקו: {removed_rows}")

# הצגת ה-ID שנמחקו
print("ID של השורות שנמחקו בגלל name לא תקין:")
print(deleted_ids)



In [ ]:
# מיזוג עמודות של 'name' ו 'original_name' לעמודה חדשה שנקראת 'final_name' ומחיקת שתי העמודות המקוריות
import re

# --- פונקציה למיזוג שמות ---
def merge_names(row):
    name = row['name']
    original = row['original_name']

    if pd.isna(name) and pd.isna(original):
        return None
    elif pd.isna(name):
        return original
    elif pd.isna(original):
        return name
    elif name == original:
        return name
    else:
        return f"{name} / {original}"

# --- יוצרים את העמודה המאוחדת ---
df_unique['final_name'] = df_unique.apply(merge_names, axis=1)

# --- ניקוי ושיפוץ ה-final_name ---
df_unique['final_name'] = df_unique['final_name'].str.strip()                             # הסרת רווחים מיותרים
df_unique['final_name'] = df_unique['final_name'].str.lower()                             # המרה לאותיות קטנות
df_unique['final_name'] = df_unique['final_name'].str.replace(r'[^\w\s]', '', regex=True)  # הסרת תווים מיוחדים

# --- המרת העמודה לסוג string של pandas ---
df_unique['final_name'] = df_unique['final_name'].astype('string')

# --- מחיקת העמודות המקוריות ---
df_unique = df_unique.drop(columns=['name', 'original_name'])

# --- בדיקה של 20 השורות הראשונות ---
print(df_unique[['final_name']].head(20))

# --- בדיקת סוג הנתונים ---
print("\nData type of final_name:", df_unique['final_name'].dtype)


# Repeated columns with the same information


In [ ]:
# Repeated columns with the same information
# First - take a look at language

# מציג את חמש השורות הראשונות של העמודות הרצויות
print(df_unique[['original_language', 'languages', 'origin_country',
                 'spoken_languages', 'production_countries']].head())

In [ ]:
# Check language uniques
language_columns = ['original_language', 'languages', 'origin_country',
                    'spoken_languages', 'production_countries']

for col in language_columns:
    print(f"\n--- {col} ---")
    print(df_unique[col].unique())

In [ ]:
### Remove column!!
# Leave only one language column - "original_language", and remove the others.... Dont' remove the 'production_countries'

# רשימת העמודות לשפה שאנחנו רוצים למחוק
cols_to_drop = ['languages', 'origin_country', 'spoken_languages']

# מוחק את העמודות האלו
df_unique = df_unique.drop(columns=cols_to_drop)

# בודק את העמודות שנותרו
print(df_unique.columns)

# בודק כמה שורות נשארו
print(df_unique.shape)


# Columns with more then 50% missing values

In [ ]:
# Columns with more then 50% missing data - nulls.

# חישוב אחוז השורות החסרות בכל עמודה
missing_percent = df_unique.isnull().mean() * 100

# בחירת העמודות עם 50% או יותר חסר
columns_50pct_or_more_missing = missing_percent[missing_percent >= 50].index.tolist()

# הצגת התוצאה
print("עמודות עם יותר או שווה ל-50% ערכים חסרים:")
print(columns_50pct_or_more_missing)


In [ ]:
# הצגת 10 שורות ראשונות רק מהעמודות עם יותר מ-50% חסר
# חישוב אחוז השורות החסרות בכל עמודה
missing_percent = df_unique[columns_50pct_or_more_missing].isnull().mean() * 100

# יצירת DataFrame חדש עם כותרות מותאמות: שם העמודה + אחוז ה-NULLs
df_to_show = df_unique[columns_50pct_or_more_missing].head(10).copy()
df_to_show.columns = [f"{col} ({missing_percent[col]:.1f}% NULLs)" for col in df_to_show.columns]

# הצגת 10 השורות הראשונות עם הכותרות החדשות
print("\n10 השורות הראשונות מהעמודות עם יותר או שווה ל-50% ערכים חסרים (כולל אחוז NULLs):")
print(df_to_show)



In [ ]:
# Remove columns with more then 50% nulls, leave the 'production_countries' for furter analysis .


# חישוב אחוז השורות החסרות בכל עמודה
missing_percent = df_unique.isnull().mean() * 100

# בחירת העמודות עם 50% או יותר חסר
columns_50pct_or_more_missing = missing_percent[missing_percent >= 50].index.tolist()

# הסרת 'production_countries' מהרשימה (לא למחוק אותה)
columns_to_drop = [col for col in columns_50pct_or_more_missing if col != 'production_countries']

# הורדת העמודות
df_unique = df_unique.drop(columns=columns_to_drop)

# הצגת העמודות שנותרו
print("העמודות אחרי המחיקה (כולל production_countries):")
print(df_unique.columns.tolist())


In [ ]:
# Check overall nulls
import missingno as msno
msno.matrix(df_unique)

# בודק את העמודות שנותרו
print(df_unique.columns)

# בודק כמה שורות נשארו
print(df_unique.shape)

# Deal with "dates".

In [ ]:
# Deal with "dates".

# ---  המרת עמודות תאריכים ל-datetime ---
df_unique['first_air_date'] = pd.to_datetime(df_unique['first_air_date'], errors='coerce')
df_unique['last_air_date'] = pd.to_datetime(df_unique['last_air_date'], errors='coerce')

# ---  חליצה של תכונות שימושיות ---

# שנה, חודש, יום של התאריך הראשון
df_unique['first_year'] = df_unique['first_air_date'].dt.year
df_unique['first_month'] = df_unique['first_air_date'].dt.month
df_unique['first_day'] = df_unique['first_air_date'].dt.day

# שנה, חודש, יום של התאריך האחרון
df_unique['last_year'] = df_unique['last_air_date'].dt.year
df_unique['last_month'] = df_unique['last_air_date'].dt.month
df_unique['last_day'] = df_unique['last_air_date'].dt.day


In [ ]:
# עבור תאריכים - נעשה עמודה חדשה
# production_length
# ונשאיר רק את first_year ו last_year


import numpy as np

# נניח שכבר קיימות NaN במקום ערכים חסרים
# חישוב production_length עם NaN
df_unique['production_length'] = df_unique['last_year'] - df_unique['first_year']

# החלפת כל ה-NaN ב-production_length ב- -1
df_unique['production_length'] = df_unique['production_length'].fillna(-1)

# אפשר גם להחליף NaN בעצמות העמודות אם רוצים
df_unique['first_year'] = df_unique['first_year'].fillna(-1)
df_unique['last_year']  = df_unique['last_year'].fillna(-1)

# מחיקת העמודות המיותרות
cols_to_drop = ['first_month', 'first_day', 'last_month', 'last_day', 'first_air_date', 'last_air_date']
df_unique = df_unique.drop(columns=[col for col in cols_to_drop if col in df_unique.columns])

# בדיקה
df_unique[['first_year', 'last_year', 'production_length']].head(10)




In [ ]:
msno.matrix(df_unique)
df_unique.head(3)

# Remove uninformative column 'poster_path'

In [ ]:
# עמודה שנשארה והיא לא אינפורמטיבית שנוציא אותה
# מחיקת העמודה 'poster_path'
df_unique = df_unique.drop(columns=['poster_path'])

# הצגת רשימת העמודות אחרי המחיקה
print("רשימת העמודות אחרי מחיקה:")
print(df_unique.columns.tolist())


In [ ]:

# ניצור טבלת סיכום מקיפה לכל עמודה
summary = pd.DataFrame({
    'non_null_count': df_unique.notnull().sum(),
    'null_percent': df_unique.isnull().mean() * 100,
    'num_unique': df_unique.nunique(),
    'data_type': df_unique.dtypes,
}).sort_values(by='null_percent', ascending=False)

print(summary)

# Try to nerrow uniques

In [ ]:
# ערכים ייחודיים בכל עמודה
# רשימת העמודות
columns_to_check = ['type', 'in_production', 'genres', 'status', 'production_countries', 'networks', 'overview']

# הדפסה מסודרת של הערכים הייחודיים עם שם העמודה
for col in columns_to_check:
    print(f"ערכים ייחודיים בעמודה '{col}':")
    print(df_unique[col].unique())
    print("\n" + "-"*50 + "\n")

# מספר הערכים הייחודיים
print("Number of unique values in 'type':", df_unique['type'].nunique())
print("Number of unique values in 'in_production':", df_unique['in_production'].nunique())
print("Number of unique values in 'genres':", df_unique['genres'].nunique())
print("Number of unique values in 'status':", df_unique['status'].nunique())
print("Number of unique values in 'production_countries':", df_unique['production_countries'].nunique())
print("Number of unique values in 'networks':", df_unique['networks'].nunique())

In [ ]:
# Show all columns
pd.set_option('display.max_columns', None)
print(df_unique.head(10))

In [ ]:
# clean text for editing: remove spaces, capital letters...
# בחירת כל העמודות הטקסטואליות
text_cols = df_unique.select_dtypes(include=['object', 'string']).columns.tolist()
print("עמודות טקסטואליות לניקוי:", text_cols)

# פונקציה לניקוי מחרוזות
def clean_text(val):
    if pd.isnull(val):
        return val  # להשאיר NaN כפי שהוא
    val = str(val).strip().lower()  # הסרת רווחים והמרה לאותיות קטנות
    # ניקוי רווחים סביב פסיקים (לערכים שמופרדים בפסיקים)
    val = ','.join([v.strip() for v in val.split(',')])
    return val

# מחיקת פסיקים מיותרים ורווחים והמרה לאותיות קטנות לכל העמודות הטקסטואליות
for col in text_cols:
    df_unique[col] = df_unique[col].apply(clean_text)

# הצגת דוגמה 10 השורות הראשונות אחרי הניקוי
print(df_unique[text_cols].head(10))

In [ ]:
# Look for uniques in columns: 'production_countries' and 'networks'
# ערכים ייחודיים בכל עמודה
print(df_unique['production_countries'].unique())
print(df_unique['networks'].unique())

# מספר הערכים הייחודיים

print("Number of unique values in 'production_countries':", df_unique['production_countries'].nunique())
print("Number of unique values in 'networks':", df_unique['networks'].nunique())


In [ ]:
# Look for uniques in column: 'episode_run_time'

df_unique['episode_run_time'].unique()

In [ ]:
# Max and Min uniques values in 'episode_run_time'
# מקסימום ומינימום של הערכים הייחודיים
unique_times = df_unique['episode_run_time'].dropna().unique()
min_time = unique_times.min() if hasattr(unique_times, 'min') else min(unique_times)
max_time = unique_times.max() if hasattr(unique_times, 'max') else max(unique_times)

print(f"Minimum episode run time: {min_time} minutes")
print(f"Maximum episode run time: {max_time} minutes")

# ספירה של הערכים 0
num_zeros = (df_unique['episode_run_time'] == 0).sum()
print(f"Number of 0 values in episode_run_time: {num_zeros}")

In [ ]:
# Check df
summary = pd.DataFrame({
    'null_percent': df_unique.isnull().mean() * 100,
    'num_unique': df_unique.nunique(),
    'data_type': df_unique.dtypes
}).sort_values(by='null_percent', ascending=False)

print(summary)

## Changing strings/objects into category

In [ ]:
# Lets start with the easy columns that have short limitied uniques
categorical_cols = ['original_language', 'type', 'status', 'production_countries', 'networks', 'genres']
for col in categorical_cols:
    df_unique[col] = df_unique[col].astype('category')


In [ ]:
summary = pd.DataFrame({
    'null_percent': df_unique.isnull().mean() * 100,
    'num_unique': df_unique.nunique(),
    'data_type': df_unique.dtypes
}).sort_values(by='null_percent', ascending=False)

print(summary)

In [ ]:
df_unique['production_countries'].value_counts()

In [ ]:
# A better overview of 'production_countries' uniques

# מאפשר הדפסת כל הערכים בעמודות ללא קיצוץ
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# סופרים את מספר ההופעות של כל ערך בעמודה production_countries
value_counts = df_unique['production_countries'].value_counts()

# הופך את זה ל-DataFrame להצגה מסודרת
value_counts_df = value_counts.reset_index()
value_counts_df.columns = ['production_countries', 'count']

# מציג את כל הערכים עם מספר ההופעות
print(value_counts_df)

# בסיום, אפשר להחזיר להגדרות ברירת המחדל
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')




In [ ]:
# Uniques in 'type'
df_unique['type'].value_counts()


In [ ]:
# Uniques in 'status'

df_unique['status'].value_counts()

In [ ]:
# A better overview of 'original_language' uniques

# מאפשר הדפסת כל הערכים בעמודות ללא קיצוץ
pd.set_option('display.max_rows', None)  # מציג כל השורות
pd.set_option('display.max_columns', None)  # מציג כל העמודות

# מציג את כל הערכים ושכיחויות
print(df_unique['original_language'].value_counts())

# בסיום, אפשר להחזיר להגדרות ברירת המחדל
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_columns')

In [ ]:
# Unit 'original_language' uniques that are under 10 counts

# שפות מתחת ל 10 הפכו להיות OTHER
# עבור original_language

# 1️⃣ ניקוי רווחים ותווים מיותרים בעמודת השפה
df_unique['original_language'] = df_unique['original_language'].astype(str).str.strip().str.lower()

# 2️⃣ ספירת כל הערכים בעמודה
language_counts = df_unique['original_language'].value_counts()

# 3️⃣ זיהוי שפות שמופיעות פחות מ-10 פעמים
rare_languages = language_counts[language_counts < 10].index

# 4️⃣ החלפה של הערכים הנדירים ב-'other' ישירות בעמודה הקיימת
df_unique['original_language'] = df_unique['original_language'].apply(
    lambda x: 'other' if x in rare_languages else x
)

# 5️⃣ הצגת התוצאות
print("התפלגות השפות לאחר איחוד ערכים נדירים:")
print(df_unique['original_language'].value_counts().sort_values(ascending=False))


In [ ]:
# A better overview of 'overview' uniques

df_unique['overview'].head()

In [ ]:
# Clean text - spaces and other - in 'overview. For further editing
df_unique['overview'] = df_unique['overview'].apply(
    lambda x: x.strip().lower() if isinstance(x, str) else x
)
print(df_unique['overview'].head(20))

In [ ]:
# Check results
df_unique['networks'].value_counts()
print(df_unique['networks'].unique())

In [ ]:
# # A better overview of 'networks' uniques

# מאפשר הדפסת כל הערכים בעמודות ללא קיצוץ
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# סופרים את מספר ההופעות של כל ערך בעמודה networks
value_counts = df_unique['networks'].value_counts()

# הופכים ל-DataFrame עם עמודות: שם הרשת ומספר ההופעות
value_counts_df = value_counts.reset_index()
value_counts_df.columns = ['network', 'count']

# מציגים את כל הערכים עם מספר ההופעות
print(value_counts_df)

# בסיום, אפשר להחזיר להגדרות ברירת המחדל
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')


In [ ]:
# Unit 'networks' uniques that are under 10 counts

# 1️⃣ סופרים את מספר ההופעות של כל רשת
network_counts = df_unique['networks'].value_counts()

# 2️⃣ מזהים את הערכים שמופיעים פחות מ-10 פעמים
rare_networks = network_counts[network_counts < 10].index.tolist()

# 3️⃣ מחליפים את הערכים הנדירים ב-'OTHER'
df_unique['networks'] = df_unique['networks'].apply(lambda x: 'OTHER' if x in rare_networks else x)

# 4️⃣ בדיקה: השכיחויות לאחר האיחוד
print("שכיחויות הערכים לאחר איחוד נדירים ל-OTHER:")
print(df_unique['networks'].value_counts())


In [ ]:
# Explore 'genres' uniques

df_unique['genres'].value_counts()
print(df_unique['genres'].unique())

In [ ]:
# explore 'genres' unique and clean spaces... for futher editing
# ניקוי ואיחוד ז'אנרים ותיאור אחיד של מילים
# פונקציה לניקוי ולמיון הז'אנרים
def clean_genres(val):
    if pd.isna(val) or val.strip() == '':
        return val
    # פיצול לפי פסיקים, הסרת רווחים, המרה לאותיות קטנות
    items = [x.strip().lower() for x in val.split(',')]
    items.sort()  # מיון אלפביתי
    return ','.join(items)  # חיבור חזרה

# יישום הפונקציה על כל העמודה הקיימת
df_unique['genres'] = df_unique['genres'].apply(clean_genres)

# הצגת השכיחויות של הערכים לאחר הניקוי
print(df_unique['genres'].value_counts().head(100))

# סיכום: כמה ערכים ייחודיים עכשיו בעמודה
num_unique_genres = df_unique['genres'].nunique()
print(f"\nסה\"כ ערכים ייחודיים בעמודה 'genres' לאחר הניקוי: {num_unique_genres}")


In [ ]:
# Unit 'genres' uniques that are under 10 counts

# 1️⃣ ספירת כל הערכים בעמודת genres
counts = df_unique['genres'].value_counts()

# 2️⃣ זיהוי הערכים שמופיעים פחות מ-10 פעמים
to_replace = counts[counts < 10].index.tolist()

# 3️⃣ החלפת הערכים הנדירים ב-'other'
df_unique['genres'] = df_unique['genres'].apply(
    lambda x: 'other' if x in to_replace else x
)

# 4️⃣ בדיקה: הצגת ההתפלגות לאחר האיחוד
value_counts = df_unique['genres'].value_counts()
print(value_counts)

# 5️⃣ הצגת סיכום מספר הערכים הייחודיים לאחר האיחוד
print(f"\nסה\"כ ערכים ייחודיים בעמודה 'genres' לאחר האיחוד: {df_unique['genres'].nunique()}")


In [ ]:
# To have an overview of missing data and types of data

# ניצור טבלת סיכום מקיפה לכל עמודה
summary = pd.DataFrame({
    'non_null_count': df_unique.notnull().sum(),
    'null_percent': df_unique.isnull().mean() * 100,
    'num_unique': df_unique.nunique(),
    'data_type': df_unique.dtypes,
}).sort_values(by='null_percent', ascending=False)

print(summary)

In [ ]:
# רשימת העמודות שיישמרו כ-object
keep_object = ['overview', 'final_name']

# המרה של כל שאר העמודות מסוג object ל-category
for col in df_unique.select_dtypes(include=['object']).columns:
    if col not in keep_object:
        df_unique[col] = df_unique[col].astype('category')

# בדיקה
# ניצור טבלת סיכום מקיפה לכל עמודה
summary = pd.DataFrame({
    'non_null_count': df_unique.notnull().sum(),
    'null_percent': df_unique.isnull().mean() * 100,
    'num_unique': df_unique.nunique(),
    'data_type': df_unique.dtypes,
}).sort_values(by='null_percent', ascending=False)

print(summary)


# EDA

In [ ]:
!pip install autoviz

In [ ]:
!pip install textblob
!python -m textblob.download_corpora


### EDA reports

In [ ]:
# AutoViz report - Popularity as Target


%matplotlib inline
from autoviz.AutoViz_Class import AutoViz_Class

AV = AutoViz_Class()

df_auto = AV.AutoViz(
    filename="",
    dfte=df_unique,
    depVar="popularity",
    sep=",",
    chart_format="png",   # תומך יותר מכל
    max_rows_analyzed=30000,
    verbose=2             # מציג מידע נוסף בהרצה
)


In [ ]:
# ydata-profiling report

# התקנה (אם טרם הותקן)
!pip install ydata-profiling --quiet

# ייבוא הספרייה
from ydata_profiling import ProfileReport
import pandas as pd

In [ ]:


# יצירת דו"ח
profile = ProfileReport(df_unique, title="EDA Report - Dataset Overview", explorative=True)

# להצגה ישירה במחברת (Jupyter / Colab)
profile.to_notebook_iframe()

# או לשמירה לקובץ HTML
profile.to_file("EDA_report.html")

print("✅ דו\"ח EDA נוצר ונשמר כ-'EDA_report.html'")


In [ ]:
profile.to_file("EDA_report.html")

In [ ]:

from google.colab import files
files.download("EDA_report.html")

# Data cleansing

In [ ]:
# Dealing with missing values
# ניצור טבלת סיכום מקיפה לכל עמודה
summary = pd.DataFrame({
    'non_null_count': df_unique.notnull().sum(),
    'null_percent': df_unique.isnull().mean() * 100,
    'num_unique': df_unique.nunique(),
    'data_type': df_unique.dtypes,
}).sort_values(by='null_percent', ascending=False)

print(summary)

In [ ]:
!pip install fancyimpute

In [ ]:
# MICE - to fill missing values in 'overview'

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from fancyimpute import IterativeImputer

# -----------------------------
# 0️⃣ מילוי חוסרים ב-overview
# -----------------------------
# ממלא NaN
df_unique['overview'] = df_unique['overview'].fillna("unknown")
# מסיר רווחים וממירה לאותיות קטנות
df_unique['overview'] = df_unique['overview'].apply(lambda x: x.strip().lower() if isinstance(x, str) else x)
# ממלא מחרוזות ריקות ב-"unknown"
df_unique.loc[df_unique['overview'] == "", 'overview'] = "unknown"

# -----------------------------
# 1️⃣ יצירת TF-IDF מה-overview (למטרת MICE בלבד)
# -----------------------------
vectorizer = TfidfVectorizer(max_features=100)
overview_tfidf = vectorizer.fit_transform(df_unique['overview']).toarray()
overview_df = pd.DataFrame(overview_tfidf, columns=[f"word_{i}" for i in range(overview_tfidf.shape[1])])

# -----------------------------
# 2️⃣ המרת קטגוריות ל-numeric
# -----------------------------
categorical_cols = ['genres', 'networks', 'production_countries', 'type', 'status', 'adult']
le_dict = {}

for col in categorical_cols:
    if pd.api.types.is_categorical_dtype(df_unique[col]):
        df_unique[col] = df_unique[col].cat.add_categories(["Unknown"])
    df_unique[col] = df_unique[col].fillna("Unknown")

    le = LabelEncoder()
    df_unique[col] = le.fit_transform(df_unique[col])
    le_dict[col] = le

# -----------------------------
# 3️⃣ הכנת דאטה ל-MICE (numeric בלבד + TF-IDF)
#    ✅ הוצאת id כדי לא לפגוע ב-MICE
# -----------------------------
numeric_cols = df_unique.select_dtypes(include=['int64', 'float64']).columns.tolist()
numeric_cols = [col for col in numeric_cols if col != 'id']  # <-- כאן הוספנו את התיקון

df_for_mice = pd.concat([df_unique[numeric_cols], overview_df], axis=1)

# -----------------------------
# 4️⃣ ריצת MICE
# -----------------------------
imp = IterativeImputer(max_iter=10, random_state=0)
df_filled_array = imp.fit_transform(df_for_mice)
df_filled = pd.DataFrame(df_filled_array, columns=df_for_mice.columns)

# -----------------------------
# 5️⃣ החזרת קטגוריות למצב טקסט
# -----------------------------
for col in categorical_cols:
    le = le_dict[col]
    df_filled[col] = df_filled[col].round().astype(int)
    df_filled[col] = le.inverse_transform(df_filled[col])

# -----------------------------
# 6️⃣ החזרת overview המקורי (שכבר מלא בהשלמות)
# -----------------------------
df_filled['overview'] = df_unique['overview']

# -----------------------------
# 7️⃣ מחיקת עמודות TF-IDF
# -----------------------------
tfidf_cols = [col for col in df_filled.columns if col.startswith("word_")]
df_filled.drop(columns=tfidf_cols, inplace=True)

# -----------------------------
# 8️⃣ החזרת כל העמודות הטקסטואליות המקוריות שלא עברו MICE
# -----------------------------
non_numeric_cols = df_unique.select_dtypes(exclude=['int64', 'float64']).columns.tolist()
for col in non_numeric_cols:
    if col != 'overview':  # overview כבר הוספנו
        df_filled[col] = df_unique[col]

# -----------------------------
# ✅ החזרת עמודת id כמו שהיא
# -----------------------------
df_filled['id'] = df_unique['id']

# -----------------------------
# 9️⃣ בדיקה
# -----------------------------
print("מספר ערכים חסרים אחרי כל התהליך:")
print(df_filled.isnull().sum())


# new dataframe - df_filled

In [ ]:
# File check
df_filled.info()

In [ ]:


from google.colab import files

df_filled.to_csv('df_new_GitHub.csv', index=False)
files.download('df_new_GitHub.csv')



# Continue in second file

In [ ]:
# For GitHub upload
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import nbformat

# Path ל-notebook הנוכחי
path = '/content/drive/MyDrive/Project TV show popularity/advance project/More advanced project/For GitHub/Upload to GitHub/GitHub_1_TV_show_popularity_part_one_upload.ipynb'  # שנה לפי הנתיב שלך



# קריאה ועריכה של ה-notebook
nb = nbformat.read(path, as_version=4)

# ניקוי metadata בעייתית
if "widgets" in nb.metadata:
    del nb.metadata["widgets"]
if "colab" in nb.metadata:
    del nb.metadata["colab"]
if "celltoolbar" in nb.metadata:
    del nb.metadata["celltoolbar"]

# שמירה מחדש
nbformat.write(nb, path)
print("✅ Notebook cleaned and ready for GitHub!")